In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import db_helper
import datetime
import networkx as nx
from datetime import datetime
from gensim.models import Word2Vec
import numpy as np
import matplotlib.pyplot as plt
import importlib
import graph_constructor as GC
import n2v as N2V
import utils
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [145]:
import gc
gc.collect()

33958313

In [4]:
PROJECT_PATH='/home/hmenn/Workspace/CSE496_Node2Vec_2018'
MONTH_TAG_EN="month"
YEAR_TAG_EN="year"
DAY_TAG_EN="day"

MONTH_NAMES_EN = {1:"JAN",2:"FEB",3:"MAR",4:"APR",
          5:"MAY",6:"JUN",7:"JUL",8:"AUG",
         9:"SEP",10:"OCT",11:"NOV",12:"DEC"}

MONTH_TAG_TR="AY"
YEAR_TAG_TR="YIL"
DAY_TAG_TR="GUN"

MONTH_NAMES_TR = {1:"OCA",2:"SUB",3:"MAR",4:"NIS",
          5:"MAY",6:"HAZ",7:"TEM",8:"AGU",
         9:"EYL",10:"EKI",11:"KAS",12:"ARA"}

If you don't have json data, read data from db and then convert them into json to easy load-save. Reading data from db is not useful.

In [28]:
db = db_helper.DBHelper()
db.connectMysql(user="root",passwd="Hasan5695*",db="SocialMediaDB")
tweets = db.getTweetsFromMysql("TweetsShort")
db_helper.saveTweetsAsJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json', tweets)

In [5]:
tweets = db_helper.getTweetsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/tweets.json')

In [21]:
allFriendships = db_helper.getAllFriendsFromJson(PROJECT_PATH+'/dataset/SocialMediaDataset/friends.json')

In [136]:
print('TotalTweet#:',len(tweets))
print(tweets[:2])

TotalTweet#: 93753
[############ Tweet: ############ 
SendByScreenName:cakrr_ 
UserID:u1623755515 
TweetID:t763141405766250496 
InReplyToStatusID:t763141168771244032 
InReplyToUserID:u757176735158923264 
Date:2016-08-10 01:34:24 
Text:@Wesleyoloji10 OGLAK HAZİNE ARIYOR AQ:das:asd 
, ############ Tweet: ############ 
SendByScreenName:selcukarici 
UserID:u154236788 
TweetID:t763577912696135680 
InReplyToStatusID:t-1 
InReplyToUserID:u-1 
Date:2016-08-11 06:28:55 
Text:Sn.CB'nımızın çağrısıyla 3 banka konut kredisi faiz oranlarını ℅1'in altına çekti..Bu olumlu bir adım.

Ancak şişen piyasa nasıl düşecek? 
]


In [131]:
print(allFriendships[:2])
userFriends = [('u100341775',x['FriendID'], {'weight':1}) for x in allFriendships[:] if x['UserID']=='u100341775']
print(userFriends)

[{'UserID': 'u100341775', 'FriendID': 'u1014141614'}, {'UserID': 'u100341775', 'FriendID': 'u120840897'}]
[('u100341775', 'u1014141614', {'weight': 1}), ('u100341775', 'u120840897', {'weight': 1}), ('u100341775', 'u1494034783', {'weight': 1}), ('u100341775', 'u154236788', {'weight': 1}), ('u100341775', 'u155534196', {'weight': 1}), ('u100341775', 'u190171522', {'weight': 1}), ('u100341775', 'u219248343', {'weight': 1}), ('u100341775', 'u2547943374', {'weight': 1}), ('u100341775', 'u25506738', {'weight': 1}), ('u100341775', 'u272613386', {'weight': 1}), ('u100341775', 'u285685164', {'weight': 1}), ('u100341775', 'u3228821009', {'weight': 1}), ('u100341775', 'u41931916', {'weight': 1}), ('u100341775', 'u425819395', {'weight': 1}), ('u100341775', 'u461858710', {'weight': 1}), ('u100341775', 'u497735542', {'weight': 1}), ('u100341775', 'u61189897', {'weight': 1}), ('u100341775', 'u63110096', {'weight': 1}), ('u100341775', 'u737302248338952193', {'weight': 1})]


In [142]:
def create_all_graph(tweets):
    G1 = nx.DiGraph()
    G1.add_node(YEAR_TAG_TR)
    G1.add_node(MONTH_TAG_TR)
    G1.add_node(DAY_TAG_TR)

    prevYear = None
    prevMonth = None
    prevDay = None

    prevYear = tweets[0].date.year
    prevMonth = tweets[0].date.month
    prevDay = tweets[0].date.day

    for tweet in tweets:
        G1.add_edge(YEAR_TAG_TR,str(tweet.date.year),weight=1.0)
        
        if(prevYear < tweet.date.year):
            G1.add_edge(str(prevYear),str(tweet.date.year),weight=1.0)
            prevYear = tweet.date.year
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
          
        monthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[tweet.date.month],year=tweet.date.year)
        G1.add_edge(str(tweet.date.year),monthFormat,weight=1.0)
        
        if(prevMonth < tweet.date.month):
            prevMonthFormat = "{month}.{year}".format(month=MONTH_NAMES_TR[prevMonth],year=tweet.date.year)
            G1.add_edge(prevMonthFormat,monthFormat,weight=1.0)
            prevMonth = tweet.date.month
            prevDay = tweet.date.day
        
        #G1.add_edge(MONTH_TAG,monthFormat,weight=1.0)
        fullDateFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
        if prevDay < tweet.date.day:
            prevFullDateFormat = "{day}.{month}.{year}".format(day=prevDay,month=tweet.date.month,year=tweet.date.year)
            G1.add_edge(prevFullDateFormat,fullDateFormat,weight=1.0)
            prevDay = tweet.date.day
        
        # tweet -> date
        G1.add_edge(tweet.tweetID,fullDateFormat,weight=1.0)
        G1.add_edge(monthFormat,fullDateFormat,weight=1.0)
                
        # if user will be added first time, add friend connections
        if not G1.has_node(tweet.userID):
            userFriends = [(x['UserID'],x['FriendID'], {'weight':1.0}) for x in allFriendships if x['UserID']==tweet.userID]
            #print('User:{userID} was added. User has {friendCount} friend.'.format(userID=tweet.userID,friendCount=len(userFriends)))

            # add friendship connections
            G1.add_edges_from(userFriends)
            
        # user -> tweet connections were established
        G1.add_edge(tweet.userID, tweet.tweetID,weight=1.0)
                 
        # if tweet replied and db has actuaal(replied tweet, add to graph)
        # cevap verilen tweet db de ise, o tweete baglanti ekle
        for i in tweets:
            if i.tweetID == tweet.inReplyToStatusID:
                G1.add_edge(tweet.tweetID, tweet.inReplyToStatusID,weight=1.0)

    return G1

In [159]:
%%time
graph = create_all_graph(tweets[:3000])
print("Number of edges: ",graph.number_of_edges())
print("Number of nodes: ",graph.number_of_nodes())

Number of edges:  6582
Number of nodes:  3259
CPU times: user 853 ms, sys: 0 ns, total: 853 ms
Wall time: 852 ms


In [138]:
# save and read edge list
#nx.write_edgelist(graph, path=PROJECT_PATH+"/outputs/grid.edgelist", delimiter=":")
#graph = nx.read_edgelist(path="grid.edgelist", delimiter=":")

In [140]:
#graph.edges()
#graph['u100341775']

In [160]:
%%time
graphN2V = N2V.Graph(graph, True, 1.0, 1.0)
graphN2V.preprocess_transition_probs()
walks = graphN2V.simulate_walks(5000, 80)

Walk iteration:
500 / 5000
1000 / 5000
1500 / 5000
2000 / 5000
2500 / 5000
3000 / 5000
3500 / 5000
4000 / 5000
4500 / 5000
5000 / 5000
CPU times: user 12min 54s, sys: 3.41 s, total: 12min 57s
Wall time: 12min 58s


In [161]:
%%time
model = N2V.learn_embeddings(walks,"graphAll")

CPU times: user 17min 28s, sys: 7.7 s, total: 17min 36s
Wall time: 8min 24s


In [162]:
model.wv.most_similar('u1623755515')

[('u829741120045342726', 0.620094358921051),
 ('u253702889', 0.6142395734786987),
 ('u292822200', 0.6137721538543701),
 ('u3363862253', 0.6123930215835571),
 ('u710792618595250178', 0.6121010780334473),
 ('u757176735158923264', 0.6119275093078613),
 ('u819201027857641472', 0.6114423274993896),
 ('u838307023527768064', 0.6111672520637512),
 ('u82946124', 0.6099460124969482),
 ('u826443466649518081', 0.6081868410110474)]

### Cosine Similarities for Times

In [164]:
column_names = ['2016', 'AGU.2016', '24.8.2016', "20.8.2016", 'EYL.2016', '2.9.2016', '6.9.2016','EKI.2016','24.10.2016','26.10.2016']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,2016,AGU.2016,24.8.2016,20.8.2016,EYL.2016,2.9.2016,6.9.2016,EKI.2016,24.10.2016,26.10.2016
2016,1.000000,0.900598,0.082261,0.130266,0.812326,0.132598,0.141010,0.733549,0.174011,0.164666
AGU.2016,0.900598,1.000000,0.175179,0.211994,0.673382,0.152823,0.143122,0.614170,0.063775,0.045359
24.8.2016,0.082261,0.175179,1.000000,0.308358,0.052272,0.091863,0.143714,0.058124,0.020044,0.043140
20.8.2016,0.130266,0.211994,0.308358,1.000000,0.129036,0.098552,0.135438,0.051146,0.048997,0.066207
EYL.2016,0.812326,0.673382,0.052272,0.129036,1.000000,0.352799,0.377248,0.739354,0.352966,0.351478
2.9.2016,0.132598,0.152823,0.091863,0.098552,0.352799,1.000000,0.643052,0.269201,0.255232,0.268627
6.9.2016,0.141010,0.143122,0.143714,0.135438,0.377248,0.643052,1.000000,0.288106,0.309457,0.296526
EKI.2016,0.733549,0.614170,0.058124,0.051146,0.739354,0.269201,0.288106,1.000000,0.419417,0.404611
24.10.2016,0.174011,0.063775,0.020044,0.048997,0.352966,0.255232,0.309457,0.419417,1.000000,0.561415
26.10.2016,0.164666,0.045359,0.043140,0.066207,0.351478,0.268627,0.296526,0.404611,0.561415,1.000000


### Cosine Similarities for Users

In [165]:
column_names = ['u1623755515','u4242878596','u2335496184']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,u1623755515,u4242878596,u2335496184
u1623755515,1.000000,0.594814,0.115294
u4242878596,0.594814,1.000000,0.290579
u2335496184,0.115294,0.290579,1.000000


### Cosine Similarities for Tweets

In [166]:
column_names = ['t766739153195266049','t766738735786491905', 't766744289342808069']
matrix1 = np.stack((model.wv[column_names]))
cosines1 = cosine_similarity(matrix1)
df1 = pd.DataFrame(cosines1, columns=column_names, index=column_names)
df1

,t766739153195266049,t766738735786491905,t766744289342808069
t766739153195266049,1.000000,0.938786,0.390433
t766738735786491905,0.938786,1.000000,0.404209
t766744289342808069,0.390433,0.404209,1.000000
